In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix


In [10]:
class SpotifyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class SpotifyRankPredictor(nn.Module):
    def __init__(self, num_categories):
        super(SpotifyRankPredictor, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(8, 64),  # 첫 번째 층의 뉴런 수를 증가
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.3),  # 드롭아웃 비율 증가
            nn.Linear(64, 32),  # 중간 층 추가
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.2),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.BatchNorm1d(16),
            nn.Linear(16, num_categories)
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.layers(x)
        return self.softmax(x)

def augment_features(features, num_samples_needed):
    augmented_data = []
    feature_names = ['Danceability', 'Energy', 'Loudness', 'Speechiness',
                     'Acousticness', 'Liveness', 'Tempo', 'Duration (ms)']

    # 노이즈 범위 조정 - 더 섬세한 변화를 위해 값들을 약간 줄임
    noise_ranges = {
        'Danceability': 0.03,
        'Energy': 0.03,
        'Loudness': 0.8,
        'Speechiness': 0.015,
        'Acousticness': 0.03,
        'Liveness': 0.03,
        'Tempo': 2.0,
        'Duration (ms)': 0.03
    }

    for _ in range(num_samples_needed):
        base_sample = features[np.random.randint(len(features))]
        new_features = []

        for feat_idx, feat_name in enumerate(feature_names):
            feature = np.abs(base_sample[feat_idx])

            if feat_name == 'Loudness':
                noise = np.random.normal(0, np.abs(noise_ranges['Loudness']))
            elif feat_name == 'Tempo':
                noise = np.random.normal(0, np.abs(noise_ranges['Tempo']))
            elif feat_name == 'Duration (ms)':
                noise = np.random.normal(0, np.abs(feature * noise_ranges['Duration (ms)']))
            else:
                noise = np.random.normal(0, np.abs(feature * noise_ranges[feat_name]))

            new_value = feature + noise

            if feat_name == 'Loudness':
                new_value = np.clip(new_value, -60, 0)
            elif feat_name == 'Duration (ms)':
                new_value = max(1000, new_value)
            else:
                new_value = np.clip(new_value, 0, 1)

            new_features.append(new_value)

        augmented_data.append(new_features)

    return np.array(augmented_data)

def calculate_class_weights(categories):
    class_counts = np.bincount(categories)
    total_samples = np.sum(class_counts)
    class_weights = total_samples / (len(class_counts) * class_counts)
    return torch.FloatTensor(class_weights)

def preprocess_data_with_augmentation(df):
    X = df[['Danceability', 'Energy', 'Loudness', 'Speechiness',
            'Acousticness', 'Liveness', 'Tempo', 'Duration (ms)']].values

    def rank_to_category(rank):
        if rank <= 10:
            return 0
        elif rank <= 30:
            return 1
        elif rank <= 50:
            return 2
        elif rank <= 100:
            return 3
        else:
            return 4

    ranks = df['Highest Charting Position'].values
    categories = np.array([rank_to_category(rank) for rank in ranks])

    # 각 카테고리별 샘플 수 계산
    category_counts = np.bincount(categories)
    print("Original category counts:", category_counts)

    # 목표 샘플 수 설정 (가장 많은 클래스의 90%로 조정)
    target_samples = int(max(category_counts) * 0.9)
    print("Target samples per category:", target_samples)

    processed_features = []
    processed_categories = []

    for category in range(5):
        category_mask = categories == category
        category_features = X[category_mask]
        current_samples = len(category_features)

        if current_samples < target_samples:
            num_samples_needed = target_samples - current_samples
            augmented = augment_features(category_features, num_samples_needed)
            processed_features.append(np.vstack([category_features, augmented]))
            processed_categories.extend([category] * target_samples)
        else:
            processed_features.append(category_features)
            processed_categories.extend([category] * current_samples)

    X_combined = np.vstack(processed_features)
    categories_combined = np.array(processed_categories)

    # 특성 스케일링
    scaler = StandardScaler()  # MinMaxScaler에서 StandardScaler로 변경
    X_scaled = scaler.fit_transform(X_combined)

    # 원-핫 인코딩
    num_categories = 5
    y_encoded = np.eye(num_categories)[categories_combined]

    print("Final category counts:", np.bincount(categories_combined))

    # 클래스 가중치 계산
    class_weights = calculate_class_weights(categories_combined)

    return X_scaled, y_encoded, num_categories, class_weights

def train_model(model, train_loader, val_loader, criterion, optimizer,
                num_epochs=150, patience=15):  # 에포크 수와 patience 증가
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    criterion = criterion.to(device)  # 가중치가 적용된 criterion을 device로 이동

    best_val_loss = float('inf')
    patience_counter = 0
    train_losses = []
    val_losses = []
    best_val_accuracy = 0  # 최고 검증 정확도 추적

    for epoch in range(num_epochs):
        # 훈련 단계
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()

            # Gradient Clipping 추가
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            _, actual = torch.max(y_batch.data, 1)
            train_total += y_batch.size(0)
            train_correct += (predicted == actual).sum().item()

        train_loss /= len(train_loader)
        train_accuracy = 100 * train_correct / train_total
        train_losses.append(train_loss)

        # 검증 단계
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                _, actual = torch.max(y_batch.data, 1)
                val_total += y_batch.size(0)
                val_correct += (predicted == actual).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = 100 * val_correct / val_total
        val_losses.append(val_loss)

        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')

        # 모델 저장 조건을 검증 정확도 기준으로 변경
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            patience_counter = 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_accuracy': val_accuracy,
            }, 'best_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'Early stopping triggered! Best validation accuracy: {best_val_accuracy:.2f}%')
                break

    return train_losses, val_losses

def main():
    df = pd.read_csv('spotify_dataset.csv')
    X_scaled, y_encoded, num_categories, class_weights = preprocess_data_with_augmentation(df)

    # 데이터 분할 비율 조정 (70/15/15)
    X_train, X_temp, y_train, y_temp = train_test_split(
        X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded.argmax(axis=1)
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp.argmax(axis=1)
    )

    train_dataset = SpotifyDataset(X_train, y_train)
    val_dataset = SpotifyDataset(X_val, y_val)
    test_dataset = SpotifyDataset(X_test, y_test)

    batch_size = 64  # 배치 사이즈 증가
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    model = SpotifyRankPredictor(num_categories)

    # 가중치를 적용한 CrossEntropyLoss 사용
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    # 옵티마이저 설정 변경
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)

    # 학습률 스케줄러 추가
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max',
                                                     factor=0.5, patience=5,
                                                     verbose=True)

    train_losses, val_losses = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        num_epochs=150,
        patience=15
    )

    # 최고 성능 모델 불러오기
    checkpoint = torch.load('best_model.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    # 테스트 세트에서 평가
    test_loss = 0
    correct = 0
    total = 0
    y_pred = []
    y_true = []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs.data, 1)
            _, actual = torch.max(y_batch.data, 1)
            total += y_batch.size(0)
            correct += (predicted == actual).sum().item()
            y_pred.extend(predicted.numpy())
            y_true.extend(actual.numpy())

    accuracy = 100 * correct / total
    print(f'\nTest Accuracy: {accuracy:.2f}%')

    # 혼동 행렬 및 분류 보고서 출력
    conf_matrix = confusion_matrix(y_true, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)

if __name__ == "__main__":
    main()

Original category counts: [253 325 235 565 812]
Target samples per category: 730
Final category counts: [730 730 730 730 812]
Epoch [1/150], Train Loss: 1.5977, Train Acc: 24.66%, Val Loss: 1.5815, Val Acc: 29.76%


C:\anaconda3\envs\DeepL_yonsei\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [2/150], Train Loss: 1.5699, Train Acc: 30.12%, Val Loss: 1.5612, Val Acc: 32.44%
Epoch [3/150], Train Loss: 1.5548, Train Acc: 33.13%, Val Loss: 1.5499, Val Acc: 30.29%
Epoch [4/150], Train Loss: 1.5397, Train Acc: 34.34%, Val Loss: 1.5420, Val Acc: 31.37%
Epoch [5/150], Train Loss: 1.5292, Train Acc: 36.62%, Val Loss: 1.5380, Val Acc: 33.78%
Epoch [6/150], Train Loss: 1.5243, Train Acc: 37.45%, Val Loss: 1.5339, Val Acc: 34.32%
Epoch [7/150], Train Loss: 1.5240, Train Acc: 36.65%, Val Loss: 1.5337, Val Acc: 34.32%
Epoch [8/150], Train Loss: 1.5133, Train Acc: 37.42%, Val Loss: 1.5349, Val Acc: 34.58%
Epoch [9/150], Train Loss: 1.5139, Train Acc: 37.69%, Val Loss: 1.5307, Val Acc: 35.39%
Epoch [10/150], Train Loss: 1.5122, Train Acc: 38.16%, Val Loss: 1.5307, Val Acc: 34.32%
Epoch [11/150], Train Loss: 1.5094, Train Acc: 38.79%, Val Loss: 1.5309, Val Acc: 34.58%
Epoch [12/150], Train Loss: 1.5119, Train Acc: 37.19%, Val Loss: 1.5295, Val Acc: 32.98%
Epoch [13/150], Train Loss: 1

C:\Users\이강민\AppData\Local\Temp\ipykernel_15128\477411265.py:288: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('best_model.pth')
